# Práctica Neo4j
- Martínez Ostoa Néstor Iván 
- Bases de Datos No Estructuradas
- LCD 
- IIMAS, UNAM

In [1]:
import pandas as pd
import numpy as np

In [2]:
products_df = pd.read_csv('data/visitas_profeco.csv')
print(products_df.shape)
print(products_df.columns)
products_df.head()

(31214, 13)
Index(['ANIO', 'UNIDAD_ADMINISTRATIVA', 'NUMERO_EXPEDIENTE', 'FECHA_VISITA',
       'NOM_VERIFICADA', 'TIPO_PRODUCTO', 'DESCRIPCION_PRODUCTO',
       'RAZON_SOCIAL', 'GIRO_COMERCIAL', 'MUNICIPIO', 'ENTIDAD',
       'INCUMPLIMIENTO', 'RESULTADO'],
      dtype='object')


,ANIO,UNIDAD_ADMINISTRATIVA,NUMERO_EXPEDIENTE,FECHA_VISITA,NOM_VERIFICADA,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,RESULTADO
0,2015,AGUASCALIENTES,702/000009/2015,2015-01-05,NOM-015-SCFI-2007,MU�ECOS,EN EMPAQUE DE CARTON ESTAMPADO Y PLASTICO RIGI...,JULIO CEPEDA JUGUETERIAS S.A. DE C.V.,JUGUETERIA,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
1,2015,AGUASCALIENTES,702/000012/2015,2015-01-06,NOM-004-SCFI-2006,ROPA CABALLERO,CAMISA PARA CABALLERO EN COLOR AZUL MARINO DE ...,C&A DE MEXICO S. DE R.L.,TIENDA DEPARTAMENTAL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
2,2015,AGUASCALIENTES,702/000132/2015,2015-02-12,NOM-142-SSA1-1995,BRANDY,UNA BOTELLA BRANDY DE LA MARCA TORRES 10 CON L...,ASFEIXAS S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
3,2015,AGUASCALIENTES,702/000133/2015,2015-02-12,NOM-142-SSA1-1995,BRANDY,ENVASE DE VIDRIO CONTENIDO BRANDY DE LA MARCA ...,MOTEL VILLA MIRO- MIRO HOTELERIA S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
4,2015,AGUASCALIENTES,702/000134/2015,2015-02-12,NOM-142-SSA1-1995,BRANDY,BOTELLA DE VIDRIO CONTENIDO BRANDY 100% UVA GR...,MOTEL LUA- COMPA�IA HOTELERA PESCADORES S.A. ...,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA


In [3]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31214 entries, 0 to 31213
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ANIO                   31214 non-null  int64 
 1   UNIDAD_ADMINISTRATIVA  31214 non-null  object
 2   NUMERO_EXPEDIENTE      31214 non-null  object
 3   FECHA_VISITA           31214 non-null  object
 4   NOM_VERIFICADA         31214 non-null  object
 5   TIPO_PRODUCTO          31214 non-null  object
 6   DESCRIPCION_PRODUCTO   31214 non-null  object
 7   RAZON_SOCIAL           31214 non-null  object
 8   GIRO_COMERCIAL         31214 non-null  object
 9   MUNICIPIO              31214 non-null  object
 10  ENTIDAD                31214 non-null  object
 11  INCUMPLIMIENTO         31214 non-null  object
 12  RESULTADO              31214 non-null  object
dtypes: int64(1), object(12)
memory usage: 3.1+ MB


In [4]:
def analyze_column(col, all=False):
    l = products_df[col].unique()
    print(f"Shape: {l.shape}")
    print("--------------")
    if l.shape[0] <= 100:
        print(l)
    else:
        if all:
            print(l)
        else:
            print(l[:50])

In [5]:
analyze_column('UNIDAD_ADMINISTRATIVA')

Shape: (49,)
--------------
['AGUASCALIENTES' 'BAJA CALIFORNIA' 'BAJA CALIFORNIA SUR' 'CAMPECHE'
 'CENTRO' 'CHIAPAS' 'CHIHUAHUA' 'CHIHUAHUA SUBDELEGACION' 'CIUDAD OBREGON'
 'COAHUILA' 'COLIMA' 'DIRECCION GENERAL DE VERIFICACION Y VIGILANCIA'
 'DURANGO' 'ECATEPEC' 'GUANAJUATO' 'GUERRERO' 'HIDALGO' 'IGUALA' 'JALISCO'
 'MANZANILLO' 'MEXICALI' 'MICHOACAN' 'MORELOS' 'NAUCALPAN' 'NAYARIT'
 'NEZAHUALCOYOTL' 'NUEVO LEON' 'OAXACA' 'ORIENTE' 'PLAYA DEL CARMEN'
 'PONIENTE' 'PUEBLA' 'QUERETARO' 'QUINTANA ROO' 'REYNOSA'
 'SAN LUIS POTOSI' 'SINALOA' 'SONORA' 'SUR' 'TABASCO' 'TAMAULIPAS'
 'TAPACHULA' 'TLALNEPANTLA' 'TLAXCALA' 'TOLUCA' 'TORREON' 'VERACRUZ'
 'YUCATAN' 'ZACATECAS']


## Preprocesamiento
- Conversión a catálogos
    - Estados
    - Tiendas
    - Productos
    - Relación Productos Tiendas

### Catálogo de estados

In [6]:
states_df = pd.DataFrame({'state_name':products_df['ENTIDAD'].unique()})
states_df['state_id'] = list(range(1, states_df.shape[0]+1))
states_df = states_df[['state_id', 'state_name']]
states_df.to_csv('data/states.csv', index=False)
print(f'Número de estados: {states_df.shape[0]}')
states_df.head()

Número de estados: 32


,state_id,state_name
0,1,AGUASCALIENTES
1,2,BAJA CALIFORNIA
2,3,BAJA CALIFORNIA SUR
3,4,CAMPECHE
4,5,DISTRITO FEDERAL


### Catálogo de tiendas

In [7]:
products_df.iloc[:, 7:].head()

,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,RESULTADO
0,JULIO CEPEDA JUGUETERIAS S.A. DE C.V.,JUGUETERIA,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
1,C&A DE MEXICO S. DE R.L.,TIENDA DEPARTAMENTAL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
2,ASFEIXAS S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
3,MOTEL VILLA MIRO- MIRO HOTELERIA S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
4,MOTEL LUA- COMPA�IA HOTELERA PESCADORES S.A. ...,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA


In [19]:
def get_state_id_from(state_name):
    return states_df[states_df['state_name'] == state_name]['state_id'].values[0]

def get_additional_elements_from(stores):
    purposes = []
    state_ids = []
    for store in stores:
        e = products_df[products_df['RAZON_SOCIAL'] == store][['GIRO_COMERCIAL', 'ENTIDAD']].values
        purposes.append(e[0][0])
        state_ids.append(get_state_id_from(e[0][1]))
    return purposes, state_ids

In [21]:
store_series = products_df['RAZON_SOCIAL'].unique()
purposes, state_ids = get_additional_elements_from(store_series)
stores_df = pd.DataFrame(
    {
        'store_id': list(range(1, store_series.shape[0]+1)),
        'store_name': store_series,
        'store_purpose': purposes,
        'state_id': state_ids,
    }
)
stores_df.to_csv('data/stores.csv', index=False)
stores_df.head()

,store_id,store_name,store_purpose,state_id
0,1,JULIO CEPEDA JUGUETERIAS S.A. DE C.V.,JUGUETERIA,1
1,2,C&A DE MEXICO S. DE R.L.,TIENDA DEPARTAMENTAL,1
2,3,ASFEIXAS S.A. DE C.V.,MOTEL,1
3,4,MOTEL VILLA MIRO- MIRO HOTELERIA S.A. DE C.V.,MOTEL,1
4,5,MOTEL LUA- COMPA�IA HOTELERA PESCADORES S.A. ...,MOTEL,1


### Catálogo de productos

In [127]:
products_df.iloc[:, 5:7].head()

,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO
0,MU�ECOS,EN EMPAQUE DE CARTON ESTAMPADO Y PLASTICO RIGI...
1,ROPA CABALLERO,CAMISA PARA CABALLERO EN COLOR AZUL MARINO DE ...
2,BRANDY,UNA BOTELLA BRANDY DE LA MARCA TORRES 10 CON L...
3,BRANDY,ENVASE DE VIDRIO CONTENIDO BRANDY DE LA MARCA ...
4,BRANDY,BOTELLA DE VIDRIO CONTENIDO BRANDY 100% UVA GR...


In [130]:
def clean_entity(series):
    s = series.str.strip()
    return s

In [138]:
unique_prod_description = pd.unique(pd.Series(
    clean_entity(products_df['TIPO_PRODUCTO']) + '+++' + clean_entity(products_df['DESCRIPCION_PRODUCTO'])
))

In [142]:
products = []
descriptions = []
for prod_descr in unique_prod_description:
    d = prod_descr.split('+++')
    products.append(d[0].strip())
    descriptions.append(d[1].strip())

unique_products = pd.DataFrame(
    {
        'product_id':list(range(1, unique_prod_description.shape[0]+1)),
        'product_name':products,
        'product_description':descriptions
    }
)
unique_products.to_csv('data/products.csv', index=False)
print(unique_products.shape[0])
unique_products.head()

27983


,product_id,product_name,product_description
0,1,MU�ECOS,EN EMPAQUE DE CARTON ESTAMPADO Y PLASTICO RIGI...
1,2,ROPA CABALLERO,CAMISA PARA CABALLERO EN COLOR AZUL MARINO DE ...
2,3,BRANDY,UNA BOTELLA BRANDY DE LA MARCA TORRES 10 CON L...
3,4,BRANDY,ENVASE DE VIDRIO CONTENIDO BRANDY DE LA MARCA ...
4,5,BRANDY,BOTELLA DE VIDRIO CONTENIDO BRANDY 100% UVA GR...


### Catálogo Productos Tiendas

In [91]:
products_df.iloc[:, 5:].head()

,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,RESULTADO
0,MU�ECOS,EN EMPAQUE DE CARTON ESTAMPADO Y PLASTICO RIGI...,JULIO CEPEDA JUGUETERIAS S.A. DE C.V.,JUGUETERIA,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
1,ROPA CABALLERO,CAMISA PARA CABALLERO EN COLOR AZUL MARINO DE ...,C&A DE MEXICO S. DE R.L.,TIENDA DEPARTAMENTAL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
2,BRANDY,UNA BOTELLA BRANDY DE LA MARCA TORRES 10 CON L...,ASFEIXAS S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
3,BRANDY,ENVASE DE VIDRIO CONTENIDO BRANDY DE LA MARCA ...,MOTEL VILLA MIRO- MIRO HOTELERIA S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
4,BRANDY,BOTELLA DE VIDRIO CONTENIDO BRANDY 100% UVA GR...,MOTEL LUA- COMPA�IA HOTELERA PESCADORES S.A. ...,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA


In [170]:
def get_product_id_from(name, description):
    name = name.strip()
    description = description.strip()
    return unique_products[
        (unique_products['product_name'] == name) &
        (unique_products['product_description'] == description)
    ]['product_id'].values[0]

def get_store_id_from(name):
    return stores_df[stores_df['store_name'] == name]['store_id'].values[0]

In [171]:
for idx, row in products_df.iterrows():
    try:
        get_product_id_from(row['TIPO_PRODUCTO'], row['DESCRIPCION_PRODUCTO'])
    except:
        print(row['TIPO_PRODUCTO'], row['DESCRIPCION_PRODUCTO'])
        break

In [172]:
p = []
s = []
m = []
for idx, row in products_df.iterrows():
    product_name = row['TIPO_PRODUCTO']
    product_description = row['DESCRIPCION_PRODUCTO']
    store_name = row['RAZON_SOCIAL']
    p.append(get_product_id_from(product_name, product_description))
    s.append(get_store_id_from(store_name))
    m.append(row['INCUMPLIMIENTO'])

In [173]:
product_store_df = pd.DataFrame(
    {
        'product_id':p,
        'store_id':s,
        'missing_requirement':m
    }
)
product_store_df.to_csv('data/product_store.csv', index=False)
product_store_df.head()

,product_id,store_id,missing_requirement
0,1,1,NO SE DETECTO INCUMPLIMIENTO
1,2,2,NO SE DETECTO INCUMPLIMIENTO
2,3,3,NO SE DETECTO INCUMPLIMIENTO
3,4,4,NO SE DETECTO INCUMPLIMIENTO
4,5,5,NO SE DETECTO INCUMPLIMIENTO
